<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TP8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2021

Lina MEZDOUR

date du document  :  1/12/2022

# TP : accès parallèle, traitement sur des partitions (énoncé ETUDIANT)

exemple d'accès en parallèle aux données du site Eau de France.
[Données de piezométrie](https://hubeau.eaufrance.fr/page/api-piezometrie) 

## Préparation

Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/essai

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/BDLE/TP1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Mounted at /content/drive


['smallwordcount.txt']

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark
print("installé")

     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 57.3 MB/s 
installé


Démarrer la session spark

In [ ]:
import os
# !find /usr/local/lib/ -name "pyspark" 
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.8/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","3G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1669974701277


Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select') or r.lower().startswith('with') )
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

Utiliaires : Chronomètres

In [ ]:
#------------------------------
# Chronometre : chronoPersist2
#------------------------------
import time

# Ce chronometre garantit que chaque tuple du dataframe est lu entièrement.
# En effet il est nécessaire de lire le détail de chaque tuple avant de les 'copier' en mémoire.
def chronoPersist(df):
    df.unpersist()
    t1 = time.perf_counter()
    count = df.persist().count()
    t2 = time.perf_counter()
    df.unpersist()
    print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')

def chronoPersist2(df):
  dest = df.selectExpr("*", "1")
  t1 = time.perf_counter()
  count = dest.persist().count()
  t2 = time.perf_counter()
  dest.unpersist()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour lire', count , 'elements')
        
def chronoCount(df):
  t1 = time.perf_counter()
  count = df.count()
  t2 = time.perf_counter()
  print('durée: {:.1f} s'.format(t2 - t1), 'pour dénombrer', count , 'elements')
    
print("fonctions définies")

fonctions définies


## Accès aux données

In [ ]:
import os
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

URL pour l'accès aux datasets

In [ ]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


In [ ]:
import os
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)


# acces online aux données
# https://www.egc.asso.fr/wp-content/uploads/data_Rostrenen.csv

web_dir = PUBLIC_DATASET + "/defi_EGC_2022"

download_file(web_dir, local_dir, "points_eau.csv")
download_file(web_dir, local_dir, "mesures_recentes_2022.json")

# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/points_eau.csv save in : /local/data/points_eau.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/defi_EGC_2022/mesures_recentes_2022.json save in : /local/data/mesures_recentes_2022.json


['points_eau.csv', 'mesures_recentes_2022.json']

## Dataframe *Point* : les points d'eau

In [ ]:
point = spark.read.option("delimiter",";").option("header",True).csv(local_dir + '/points_eau.csv').persist()
point.createOrReplaceTempView("point")
display(point, 3)

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,00471X0095/PZ2013,BSS000EBLL,2.578405,50.023705,80733,SENLIS-LE-SEC,E6397010,L'Hallue à Bavelincourt,121BB01,Craie du Séno-Turonien du bassin versant de la...
1,00487X0015/S1,BSS000EECH,3.07279822338651,49.9035922409316,80413,HANCOURT,E6351408,Haute Somme à Ham,121BB01,Craie du Séno-Turonien du bassin versant de la...
2,00755X0006/S1,BSS000FHCQ,0.419809838577633,49.5555270637694,76714,LES TROIS-PIERRES,G9103020,Lézarde à Montivilliers,121AU01,Craie du Séno-Turonien du Bassin Parisien de l...


Nombre de points d'eau

In [ ]:
point.count()

18

## Requetes d'analyse

In [ ]:
%%sql
select *
from Point
where nom_commune like 'R%'

,CODE_BSS,BSS_ID,LONGITUDE,LATITUDE,CODE_INSEE_COMMUNE,NOM_COMMUNE,CODE_STATION_HYDRO,NOM_STATION_HYDRO,CODE_BDLISA,NOM_ENTITE_BDLISA
0,03124X0088/F,BSS000XFPD,-3.30717741173568,48.2312060880106,22266,ROSTRENEN,J5202110,Le Blavet à Kerien [Kerlouet],195AA03,Socle plutonique dans le bassin versant du Can...


### La liste des codes de station

In [ ]:
%%sql
select distinct nom_commune, code_bss
from Point
order by nom_commune

,nom_commune,code_bss
0,BEAUCHERY-ST-MARTIN,02603X0009/S1
1,BLACQUEVILLE,00766X0004/S1
2,BOURBERAIN,04398X0002/SONDAG
3,BUHY,01258X0020/S1
4,CELLES-SUR-PLAINE,02706X0074/S77-20
5,CORBAS,07223C0113/S
6,HALLIGNICOURT,02267X0030/S1
7,HANCOURT,00487X0015/S1
8,LES GRANDES-LOGES,01584X0023/LV3
9,LES TROIS-PIERRES,00755X0006/S1


## Accès Open data online

Example d'URL pour obtenir des données depuis le **service en ligne**:
https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=07548X0009/F&sort=desc&size=10

Les paramêtres de l'appel sont
*   *code_bss* : le code du point d'eau
*   *sort* : pour trier les données par date décroissante : sort=desc
*   *size* : le nombre de tuples retournés

In [ ]:
import urllib.request
import json

# code_bss = "07548X0009/F"
code_bss = "01516X0004/S1"
size = 20

urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&sort=desc&size={size}"

# télécharger les mesures depuis le service en ligne
with urllib.request.urlopen(urlEau) as response:
  encoding = response.info().get_content_charset('utf-8')
  mesure_json = response.read().decode(encoding)

  # stocker les mesures dans un fichier
  with open(local_dir + '/mesure.json', 'w') as outfile:
    outfile.write(mesure_json)

# Lire dans Spark le fichier contenant les mesures.  Spark infère le schéma json s'il n'est pas donné
data1 = spark.read.json(local_dir + '/mesure.json')
data1.printSchema()

root
 |-- api_version: string (nullable = true)
 |-- count: long (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code_bss: string (nullable = true)
 |    |    |-- code_continuite: string (nullable = true)
 |    |    |-- code_nature_mesure: string (nullable = true)
 |    |    |-- code_producteur: string (nullable = true)
 |    |    |-- date_mesure: string (nullable = true)
 |    |    |-- mode_obtention: string (nullable = true)
 |    |    |-- niveau_nappe_eau: double (nullable = true)
 |    |    |-- nom_continuite: string (nullable = true)
 |    |    |-- nom_nature_mesure: string (nullable = true)
 |    |    |-- nom_producteur: string (nullable = true)
 |    |    |-- profondeur_nappe: double (nullable = true)
 |    |    |-- qualification: string (nullable = true)
 |    |    |-- statut: string (nullable = true)
 |    |    |-- timestamp_mesure: long (nullable = true)
 |    |    |-- urn_bss: string (nullable = true)
 |-- f

### données : les mesures de points d'eau

In [ ]:
data2 = data1.selectExpr("explode(data) as item")\
  .selectExpr("item.code_bss as code",
           "item.date_mesure as jour",
           "item.profondeur_nappe as profonfeur",
           "item.niveau_nappe_eau as niveau",
           "from_unixtime(item.timestamp_mesure/1000, 'hh:mm') as heure")\
  .orderBy(desc(col('jour')), desc(col('heure')))

data2.persist()
data2.printSchema()
display(data2, 10)

root
 |-- code: string (nullable = true)
 |-- jour: string (nullable = true)
 |-- profonfeur: double (nullable = true)
 |-- niveau: double (nullable = true)
 |-- heure: string (nullable = true)



,code,jour,profonfeur,niveau,heure
0,01516X0004/S1,2022-11-28,20.44,35.24,12:00
1,01516X0004/S1,2022-11-27,20.44,35.24,09:00
2,01516X0004/S1,2022-11-26,20.45,35.23,07:00
3,01516X0004/S1,2022-11-25,20.44,35.24,12:00
4,01516X0004/S1,2022-11-24,20.44,35.24,12:00
5,01516X0004/S1,2022-11-23,20.42,35.26,08:00
6,01516X0004/S1,2022-11-22,20.42,35.26,12:00
7,01516X0004/S1,2022-11-21,20.41,35.27,03:00
8,01516X0004/S1,2022-11-20,20.43,35.25,05:00
9,01516X0004/S1,2022-11-19,20.43,35.25,12:00


# Exercice 1 : Accès en parallèle aux mesures de tous les points d'eau

## Question 1 : Accès parallèle

Pour toutes les stations (code_bss) de la table Point. Accéder en parallèle au site pour récupérer les **10 dernières mesures de niveau d'eau**.
Le résultat est un seul dataframe Mesure(code,niveau,date,time) contenant toutes les mesures de tous les points d'eau. Rendre persistant le dataframe Mesure.

### 1a)  get_mesure
Définir la fonction get_mesure(code, nb_valeurs) avec code étant un code de station et nb_valeurs étant le nombre de mesures à récupérer.

Le type de la valeur retournée est un tableau de tuples. Un tuple contient la date, la profondeur de la nappe, le niveau d'eau et l'estampille (timestamp) de la mesure.


In [ ]:
def get_mesure(code, nb_valeurs):
  print(code, nb_valeurs)
  urlEau = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code}&sort=desc&size={nb_valeurs}"

  with urllib.request.urlopen(urlEau) as response:
    encoding = response.info().get_content_charset('utf-8')
    texte = response.read().decode(encoding)
    obj = json.loads(texte)
    data = obj['data'] #a list
    tab = [ (i["date_mesure"], i["profondeur_nappe"], i["niveau_nappe_eau"], i["timestamp_mesure"]) for i in data]
    return tab #an array of tuples

# Exemple d'invocation de la fonction
code = '01516X0004/S1'
print("Les 5 dernières mesures de la station", code)
print(get_mesure(code, 5))

Les 5 dernières mesures de la station 01516X0004/S1
01516X0004/S1 5
[('2022-11-28', 20.44, 35.24, 1669593600000), ('2022-11-27', 20.44, 35.24, 1669539600000), ('2022-11-26', 20.45, 35.23, 1669446000000), ('2022-11-25', 20.44, 35.24, 1669334400000), ('2022-11-24', 20.44, 35.24, 1669248000000)]



Déclarer la fonction comme étant une fonction UDF pour spark

In [ ]:
# syntaxe simplifiée, sans accent dans les noms de champs
# tableau de tuples(date, profondeur, niveau, estampille)
type_resultat = "array< struct<date:string, profondeur:float, niveau:float, estampille:long> >"

spark.udf.register("get_mesure", get_mesure, type_resultat)

print("fonction get_mesure déclarée")

# Rmq, la syntaxe complète est :
# type_resultat = ArrayType(StructType([
#                             StructField('date', DateType(), False),
#                             StructField('profondeur', FloatType(), True),
#                             StructField('niveau', FloatType(), True),
#                             StructField('estampille', IntegerType(), True)]))
get_mesure_udf = udf(get_mesure, returnType=type_resultat)

fonction get_mesure déclarée


### 1b) Invoquer get_mesure en parallèle

In [ ]:
mesure = point.select(col("CODE_BSS").alias("code"), explode(get_mesure_udf(col("CODE_BSS"), lit(10))).alias("item"))\
  .selectExpr("code", "item.niveau as niveau", "item.date as date", "item.estampille as time")

#select ==> Spark, with col, and @udf/udf(fun, return_type)
#selectExpr ==> SQL (les args sans des str), avec normal SQL syntax et udf.register
#C'est qui le meilleur ?
mesure.printSchema()
display(mesure,10)

root
 |-- code: string (nullable = true)
 |-- niveau: float (nullable = true)
 |-- date: string (nullable = true)
 |-- time: long (nullable = true)



,code,niveau,date,time
0,00471X0095/PZ2013,62.880001,2022-11-28,1669658400000
1,00471X0095/PZ2013,62.880001,2022-11-27,1669557600000
2,00471X0095/PZ2013,62.889999,2022-11-26,1669428000000
3,00471X0095/PZ2013,62.900002,2022-11-25,1669363200000
4,00471X0095/PZ2013,62.910000,2022-11-24,1669251600000
5,00471X0095/PZ2013,62.910000,2022-11-23,1669240800000
6,00471X0095/PZ2013,62.919998,2022-11-22,1669114800000
7,00471X0095/PZ2013,62.930000,2022-11-21,1668988800000
8,00471X0095/PZ2013,62.939999,2022-11-20,1668909600000
9,00471X0095/PZ2013,62.950001,2022-11-19,1668816000000


## Question 2 : degré de parallélisme

L'objectif est de mesurer le bénéfice aporté par le parallélisme pour accélérer le traitement d'une requête.

On considère la requête ci-dessus qui lit les mesures d'eau à partir d'une liste de points d'eau.

### 2a) 1 seule partition

Rendre persistant la liste des points d'eau dans **une seule** partition



In [ ]:
point1Partition = point.repartition(1).persist()
point1Partition.count()

18

Mesurer la durée de la requête à l'aide du chronomètre chronoPersist()

In [ ]:
# m1partition = point1Partition.select(col("CODE_BSS").alias("code"), explode(get_mesure_udf(col("CODE_BSS"), lit(10))).alias("item"))
m1partition = point1Partition.selectExpr("code_bss", "explode(get_mesure(code_bss, 10)) as item")

chronoPersist(m1partition)

durée: 11.1 s pour lire 180 elements


### 2b) Deux partitions
Rendre persistant la liste des points d'eau dans **deux** partitions, puis mesurer la durée de la requête.

In [ ]:
point2Partitions = point.repartition(2).persist()
point2Partitions.count()

18

Mesurer la durée de la requête 

In [ ]:
m2partition = point2Partitions.selectExpr("code_bss", "explode(get_mesure(code_bss, 10)) as item")

chronoPersist(m2partition)

durée: 5.7 s pour lire 180 elements


### 2b) 4 partitions

In [ ]:
point4Partitions = point.repartition(4).persist()
print(point2Partitions.count())

m4partition = point4Partitions.selectExpr("code_bss", "explode(get_mesure(code_bss, 10)) as item")

chronoPersist(m4partition)

#Justification: We only have 2 cores

18
durée: 6.0 s pour lire 180 elements


# Exercice 2 : Traitement avec des partitions

On considère les mesures récentes

In [ ]:
mesures = spark.read.format("json").load(local_dir + '/mesures_recentes_2022.json').repartition(3).persist()
print(mesures.count())
display(mesures,5)

900


,code,date,estampille,niveau,profondeur
0,07476X0029/S,2022-11-24,1669273200,297.16,31.27
1,02603X0009/S1,2022-11-12,1668211200,128.66,15.49
2,03124X0088/F,2022-11-11,1668207600,236.45,18.70
3,00487X0015/S1,2022-11-12,1668261600,69.93,32.22
4,01584X0023/LV3,2022-11-11,1668204000,83.71,23.69


On fournit des fonctions pour afficher le contenu des partitions d'un dataframe


## Question 1 : Taille et contenu d'une partition

Expliquer le fonctionnement de showPartitions et showPartitionSize

#### Fonctions showPartitions et showPartitionSize
 
* showPartitions : affiche les _n_ premiers éléments de chaque partition
* showPartitionSize : affiche le nombre d'éléments dans chaque partition

In [ ]:
# fonction auxilliaire
def partSize(partID, iterateur):
  c=0
  suivant = next(iterateur, None)
  while suivant is not None :
    c+=1
    suivant = next(iterateur, None)
  yield (partID, c)


def showPartitionSize(df):  
  t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize)
  for (partID, nbElt) in t.collect():
    print("partition", partID, ":", nbElt, "éléments")
  print()


def showPartitions(df, N=5):
  size = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize).collectAsMap()
  
  def topN(partID, iterateur):
    c=0
    head=[]
    suivant = next(iterateur, None)
    while suivant is not None and c < N :
      c+=1
      head.append(suivant)
      suivant = next(iterateur, None)
    yield (partID, head)
    
  t = df.rdd.mapPartitionsWithIndex(topN)
  for (partID, head) in t.collect():
    print("Partition", partID, ",", size[partID], "éléments")
    for row in head:
        print(row)
    print()
    
print('showPartitions et showPartitionSize définies')

showPartitions et showPartitionSize définies


Afficher la taille de chaque partition

In [ ]:
showPartitionSize(mesures)

partition 0 : 300 éléments
partition 1 : 300 éléments
partition 2 : 300 éléments



Afficher les premiers éléments de chaque partition

In [ ]:
showPartitions(mesures)

Partition 0 , 300 éléments
Row(code='07476X0029/S', date='2022-11-24', estampille=1669273200, niveau=297.16, profondeur=31.27)
Row(code='02603X0009/S1', date='2022-11-12', estampille=1668211200, niveau=128.66, profondeur=15.49)
Row(code='03124X0088/F', date='2022-11-11', estampille=1668207600, niveau=236.45, profondeur=18.7)
Row(code='00487X0015/S1', date='2022-11-12', estampille=1668261600, niveau=69.93, profondeur=32.22)
Row(code='01584X0023/LV3', date='2022-11-11', estampille=1668204000, niveau=83.71, profondeur=23.69)

Partition 1 , 300 éléments
Row(code='00471X0095/PZ2013', date='2022-11-08', estampille=1667890800, niveau=63.06, profondeur=15.02)
Row(code='01584X0023/LV3', date='2022-10-24', estampille=1666641600, niveau=83.51, profondeur=23.89)
Row(code='01584X0023/LV3', date='2022-11-01', estampille=1667343600, niveau=83.18, profondeur=24.22)
Row(code='02267X0030/S1', date='2022-10-10', estampille=1665421200, niveau=133.22, profondeur=2.87)
Row(code='07476X0029/S', date='2022-10

## Question 2 : Nombre de mesures par point d'eau

En vous inspirant des fonctions topN et showPartitions définir une fonction qui détermine le nombre de mesures de chaque point d'eau dans chaque partition. Le résultat est (numP, code, nbMesure) avec numP étant le numéro de partition.

In [ ]:
#iterateur sur les rows de la partition
#Using a python dict is possible as well (and faster)
def nbMesure(partID, iterateur):
    c=0
    suivant = next(iterateur, None)
    codes=[]
    nbMesures=[]
    while suivant is not None :
      c+=1
      if suivant.code in codes:
        for i, code in enumerate(codes):
          if code == suivant.code:
            nbMesures[i] +=1
            break
      else:
        codes.append(suivant.code)
        nbMesures.append(1)
      
      suivant = next(iterateur, None)
    
    for i in range(len(codes)):
      yield (partID, codes[i], nbMesures[i])

Invoquez la fonction que vous aurez définie

In [ ]:
# exemple d'invocation de la fonction nbMesure définie ci-dessus
nb_mesures_par_point = mesures.rdd.mapPartitionsWithIndex(nbMesure).toDF(["numP", "code", "nb"])
display(nb_mesures_par_point)

,numP,code,nb
0,0,07476X0029/S,16
1,0,02603X0009/S1,18
2,0,03124X0088/F,19
3,0,00487X0015/S1,20
4,0,01584X0023/LV3,15
5,0,04398X0002/SONDAG,15
6,0,02267X0030/S1,14
7,0,06505X0080/FORC,18
8,0,02706X0074/S77-20,17
9,0,01516X0004/S1,21


## Question 3 : Niveau d'eau maximal quotidien d'un point d'eau

Definir niveau_max_jour qui est un partitionnement de Mesure par date. 
En vous inspirant de la fonctions showPartitions,
définir une fonction qui détermine le niveau max d'un jour dans chaque partition. Le résultat est (numP, code, date, niveauMax) 

In [ ]:
! python -V

Python 3.8.15


In [ ]:

def niveau_max_jour(partID, iterateur):
  suivant = next(iterateur, None)
  codes=dict()
  while suivant is not None :
    if suivant.code in codes:
      if suivant.date in codes[suivant.code]:
        codes[suivant.code][suivant.date] = max(codes[suivant.code][suivant.date], suivant.niveau)
      else:
        codes[suivant.code][suivant.date] = suivant.niveau 
    else:
      x = dict()  
      x[suivant.date] = suivant.niveau
      codes[suivant.code]= x
    suivant = next(iterateur, None)
  
  for code, dates in codes.items():
    for date, niv in dates.items():
      yield (partID, code, date, niv)  

In [ ]:
niv_max_quotidien_par_point = mesures.rdd.mapPartitionsWithIndex(niveau_max_jour).toDF(["numP", "code", "date", "niveauMax"])
display(niv_max_quotidien_par_point)

,numP,code,date,niveauMax
0,0,07476X0029/S,2022-11-24,297.16
1,0,07476X0029/S,2022-10-13,297.49
2,0,07476X0029/S,2022-11-19,297.19
3,0,07476X0029/S,2022-10-11,297.51
4,0,07476X0029/S,2022-11-11,297.25
...,...,...,...,...
95,0,04398X0002/SONDAG,2022-11-23,226.76
96,0,04398X0002/SONDAG,2022-11-21,226.56
97,0,04398X0002/SONDAG,2022-10-12,224.46
98,0,04398X0002/SONDAG,2022-10-24,225.34


## Question 4 : Niveau moyen d'un point d'eau

En vous inspirant des fonctions topN et showPartitions définir une fonction qui détermine le niveau moyen de chaque point d'eau dans chaque partition. Le résultat est (numP, code, niveauMoyen) avec numP étant le numéro de partition.

In [ ]:
#sum has been overloaded by spark sum function ##To ask!
def niveau_moyen(partID, iterateur):
  suivant = next(iterateur, None)
  codes=dict()
  while suivant is not None :
    if suivant.code in codes:
      codes[suivant.code].append(suivant.niveau)
    else:
      codes[suivant.code]= [suivant.niveau]
    suivant = next(iterateur, None)
  
  for code, niveaux in codes.items():
      S = 0
      for n in niveaux:
        S +=n
      yield (partID, code, S/len(niveaux))

In [ ]:
niv_moyen_par_point = mesures.rdd.mapPartitionsWithIndex(niveau_moyen).toDF(["numP", "code", "niveauMoyen"])
display(niv_moyen_par_point)

,numP,code,niveauMoyen
0,0,07476X0029/S,297.306250
1,0,02603X0009/S1,128.917778
2,0,03124X0088/F,235.805789
3,0,00487X0015/S1,69.990500
4,0,01584X0023/LV3,83.676667
5,0,04398X0002/SONDAG,225.269333
6,0,02267X0030/S1,133.309286
7,0,06505X0080/FORC,168.353333
8,0,02706X0074/S77-20,310.121765
9,0,01516X0004/S1,35.264762


In [ ]:
##END